1. API 데이터 수집
→ 주기적으로 API 호출 → JSON 파일 저장

2. Unity Catalog Volume에 저장
→ DLT가 읽을 수 있도록 /Volumes/your_catalog/your_schema/your_volume에 저장

3. DLT 파이프라인 구성
- fng_bronze: 원시 JSON 로딩
- fng_silver: 파싱 및 정제

(선택) 시각화 및 분석

In [0]:
import requests
from datetime import datetime

url = "https://api.alternative.me/fng/"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()['data'][0]
    df = spark.createDataFrame([{
        "value": data['value'],
        "value_classification": data['value_classification'],
        "timestamp": int(data['timestamp']),
        "retrieved_at": datetime.now().isoformat()
    }])
    
    df.write.mode("append").json("s3a://databricks-fng-external-bucket/raw/fng/")
    print("✅ 저장 완료!")
else:
    print("❌ API 호출 실패:", response.status_code)